Example of how to create a database connection in python

In [ ]:
#conda install python-dotenv
#conda install pyodbc
#conda install pypyodbc

In [1]:
from dotenv import load_dotenv, find_dotenv
import pyodbc
import os
import pandas as pd

load_dotenv(find_dotenv("C:/Users/BoydClaire/.Renviron"))

True

**Microsoft SQL Server Database Connections**

In [2]:
# choose which database you want to connect to (assumes you have "{database}_server", "{database}_database", "{database}_username", and "{database}_password" as environmental variables)
database = "production"

# build connection string
connection_args = ["Driver={SQL Server};",
                    f"SERVER={os.environ[f'{database}_server']};",
                    f"DATABASE={os.environ[f'{database}_database']};",
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]
connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

In [3]:
# query
query = "SELECT TOP 100 * FROM REAL_PROP.REALMAST"

# read in data with query and connection
data = pd.read_sql(query, cnxn)

In [4]:
# view data
data

,REM_MNC,REM_PID,REM_PIN,REM_OWN_NAME,REM_ACCT_NUM,REM_PRCL_LOCN,REM_PRCL_LOCN_STR_PFX,REM_PRCL_LOCN_STREET,REM_PRCL_LOCN_STR_SFX,REM_PRCL_LOCN_NUM,...,REM_PRCL_LOCN_POST_DIRECTION,REM_PRCL_LOCN_PRE_DIRECTION,REM_PRCL_LOCN_STREET_TYPE,REM_PRCL_LOCN_APT_TYPE,REM_USRFLD,REM_USRFLD_DESC,REM_PRCL_LOCN_ADDRESS_ID,REM_BLOCK_SUFFIX,REM_PROPERTY_TYPE,REM_PROPERTY_TYPE_DESC
0,9000.0,1.0,,GOVERNORS ISLAND CORPORATION,1000010010,301 COMFORT COURT,,COMFORT COURT,,301.0,...,None,None,None,None,None,None,None,000,R,None
1,9000.0,2.0,,OWNER NAME,1000010101,1 LIBERTY ISLAND,,LIBERTY ISLAND,,1.0,...,None,None,None,None,None,None,None,000,R,None
2,9000.0,3.0,None,OWNER NAME,1000010201,1 ELLIS ISLAND,None,ELLIS ISLAND,None,1.0,...,None,None,None,None,None,None,None,000,R,None
3,9000.0,4.0,None,OWNER NAME,1000020001,4 SOUTH STREET,None,SOUTH STREET,None,4.0,...,None,None,None,None,None,None,None,000,R,None
4,9000.0,5.0,,OWNER NAME,1000020023,1 PIER 6,,PIER 6,,1.0,...,None,None,None,None,None,None,None,000,R,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9000.0,96.0,,MSA TWINS LTD.,1000100015,27 WHITEHALL STREET,None,WHITEHALL STREET,None,27.0,...,None,None,None,None,None,None,None,001,R,None
96,9000.0,97.0,,"BROAD STREET PLAZA, LLC",1000100016,90 BROAD STREET,,BROAD STREET,,90.0,...,None,None,None,None,None,None,None,002,R,None
97,9000.0,98.0,None,JMW RESTAURANT CORP,1000100019,25 BRIDGE STREET,None,BRIDGE STREET,None,25.0,...,None,None,None,None,None,None,None,002,R,None
98,9000.0,99.0,,CF 1 WHITEHALL LLC,1000100023,1 WHITEHALL STREET,,WHITEHALL STREET,,1.0,...,None,None,None,None,None,None,None,002,R,None


In [ ]:
cnxn.close()

**SAS Database Connections (FDW or PTS)**

In [ ]:
pyodbc.drivers()

In [ ]:
# build connection string
database = "fdw"

connection_args = ["Driver={Oracle in OraClient19Home1};",
                    f"DBQ={os.environ[f'{database}_path']};"
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]
connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

get list of tables in database

In [ ]:
cursor = cnxn.cursor()
tables_raw = cursor.tables(schema='FDW3NF')
tables = pd.DataFrame(tables_raw)
tables.columns = ['list']
cleaned_tables = pd.DataFrame(tables['list'].apply(list).to_list(), columns = ['CATALOG','SCHEMA','NAME','TYPE','SELF_REFERENCING_COL_NAME'])

In [ ]:
cleaned_tables.NAME

check if table is in the database

In [ ]:
SELECT * FROM information_schema.tables

In [ ]:
# query
# query = "SELECT * FROM VW_CAMA_REALMAST"
query = "SELECT TOP 100 * FROM FDW3NF.VW_CAMA_REALMAST"


# read in data with query and connection
data = pd.read_sql(query, cnxn)

In [ ]:
cnxn.close()